## Imports

In [ ]:
import cv2
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import re
import shutil
import os
import subprocess

## Extract info from CSV and image filenames

In [ ]:
# Define base paths
base_dir = Path.cwd().parent  # current working directory

# Define CSV and image folders relative to script
csv_dir = base_dir / "data" / "CSV"
image_dir = base_dir / "data" / "Images_RGB"

# Extract info from CSV filenames
a = []  # image IDs
b = []  # CSV file paths

for file in glob.glob(f"{csv_dir}/*"):
    b.append(file)

    bit0 = file.split("_1_HP_IM3_0_Core")[0].split(" ")[-1] # e.g.: A2
    bit1 = file.split("_1_HP_IM3_0_Core")[1].split("_")[0].split(",") # e.g. ['[1', '1', '1', '1]']
    
    p3 = f"{int(bit1[1]):02d}" # e.g. 01
    p4 = f"{int(bit1[2]):02d}" # e.g. 01
    
    a.append([bit0, p3, p4])

# Extract info from image filenames
c = []  # image IDs
d = []  # image file paths (tif)

for file in glob.glob(f"{image_dir}/*"):
    d.append(file)

    bit2 = file.split("_1_HP_IM3_0_Core")[0].split(" ")[-1]
    bit3 = file.split("_1_HP_IM3_0_Core")[1].split("_")[0].split(",")
    
    p1 = f"{int(bit3[1]):02d}"
    p2 = f"{int(bit3[2]):02d}"
    
    c.append([bit2, p1, p2])

# Sanity check: print the first parsed image result
if c and d:
    print("Example parsed image:", c[0])
    print("Example image path:", d[0])
else:
    print("No image files found.")

## Create masks by drawing circles at each detected cell location

In [ ]:
# Already have: a, b, c, d (from previous steps) and the base directory
# Define the output path for masks
mask_dir = base_dir /  "data" / "Mask"
mask_dir.mkdir(exist_ok=True) # create the folder if it doesn't exist

# Loop through CSV/image matches
for j, item in enumerate(a):  # a[j] contains ID info from CSV
    try:
        i = c.index(item)  # Match with image ID
    except ValueError:
        print(f"No match for: {item}")
        continue

    image3 = cv2.imread(d[i])
    image4 = image3.copy()

    df = pd.read_csv(b[j], sep='\t')

    print("CSV:", b[j])
    print("Image:", d[i])

    # Overlay red dots at centroids
    for _, row in df.iterrows():
        x = int(row["Centroid X µm"]) * 2
        y = int(row["Centroid Y µm"]) * 2
        cv2.circle(image3, (x, y), radius=0, color=[0, 0, 255], thickness=15)
        cv2.circle(image4, (x, y), radius=5, color=[0, 0, 200], thickness=15)

    # Blend overlays (creates composite image)
    dst_0 = cv2.addWeighted(image3, 0.5, image4, 0.5, 0)

    # Save output
    outname = os.path.splitext(os.path.basename(b[j]))[0]  + ".tiff"
    out_path = mask_dir / outname
    cv2.imwrite(str(out_path), dst_0)


## Run for Windows OS

In [ ]:
## Windows OS
import os
import subprocess

# 1. Directories and files definition
# Already have: base directory and mask_dir
os.chdir("C:/Program Files/QuPath-0.6.0")  # Update this to your actual QuPath install path
create_project_script = base_dir / "scripts" / "createproject_ki67.groovy" # Path to Groovy script that creates the QuPath project and adds images
annotate_script = base_dir / "scripts" / "annotate_ki67_cells.groovy" # Path to Groovy script that annotates the images
project_dir = base_dir / "Project" / "ki67"
project_file = project_dir / "project.qpproj"
qupath_bin = "QuPath-0.6.0 (console).exe" # Use actual QuPath  Windows console executable
geojson_dir = base_dir / "data" / "geoJSON"

# 2. Run Groovy script to create and populate the QuPath project
subprocess.run([
    qupath_bin,  
    "script",
    "--args", str(mask_dir),
    create_project_script
])

# 3. Run the annotation script, save the project (generates GeoJSON, etc.)
subprocess.run([
    qupath_bin,
    "script",
    "--save",
    "--project", project_file,
    "--args", str(geojson_dir),   # export directory here
    annotate_script
])

# 4. Optionally open the project in QuPath GUI for inspection
subprocess.Popen([
    qupath_bin,
    "--project", str(project_file)
])

In [ ]:
## Windows OS
import subprocess

# Path to the R script that performs downstream analysis (e.g., spatial stats, plotting)
Rfile = base_dir / "scripts" / "R_script_ki67.R"

# Full path to Rscript executable (update if your R version or install path differs)
rscript_exe = "C:/Program Files/R/R-4.3.1/bin/Rscript.exe"

# Run the R script with '--vanilla' to avoid loading user environment or site files
subprocess.call([rscript_exe, "--vanilla", Rfile])


## Run for MacOS

In [ ]:
## Mac OS
import os
import subprocess

# 1. Directories and files definition
# Already have: base directory and mask_dir
os.chdir("/Applications/QuPath-0.6.0-arm64.app/Contents/MacOS/")  # Adjust if QuPath installation directory (macOS .app path) is elsewhere
create_project_script = base_dir / "scripts" / "createproject_ki67.groovy" # Path to Groovy script that creates the QuPath project and adds images
annotate_script = base_dir / "scripts" / "annotate_ki67_cells.groovy" # Path to Groovy script that annotates the images
project_dir = base_dir / "Project" / "ki67"
project_file = project_dir / "project.qpproj"
qupath_bin = "./QuPath-0.6.0-arm64" # Use actual QuPath binary on macOS
geojson_dir = base_dir / "data" / "geoJSON"

# 2. Run Groovy script to create and populate the QuPath project
subprocess.run([
    qupath_bin,  
    "script",
    "--args", str(mask_dir),
    create_project_script
])

# 3. Run the annotation script, save the project (generates GeoJSON, etc.)
subprocess.run([
    qupath_bin,
    "script",
    "--save",
    "--project", project_file,
    "--args", str(geojson_dir),   # export directory here
    annotate_script
])

# 4. Optionally open the project in QuPath GUI for inspection
subprocess.Popen([
    qupath_bin,
    "--project", str(project_file)
])

In [ ]:
## MacOS
import subprocess

# Path to the R script you want to execute
Rfile = base_dir / "scripts" / "R_script_ki67.R"

# Path to Rscript binary (standard for R installations on macOS via CRAN)
rscript_exe = "/usr/local/bin/Rscript"  # or run `which Rscript` in terminal to confirm

# Run the R script with '--vanilla' for a clean R session
subprocess.call([rscript_exe, "--vanilla", Rfile])